In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import requests
from requests.auth import HTTPBasicAuth
import time
%matplotlib inline

In [2]:
NUM_ITERATIONS = 10
def avg(l):
    return float(sum(l))/(len(l))

In [35]:
query0_sparql = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX brick: <http://buildsys.org/ontologies/Brick#> 
PREFIX bf: <http://buildsys.org/ontologies/BrickFrame#> 
SELECT DISTINCT ?vav
WHERE {
    ?vav rdf:type brick:VAV .
}
"""

query0_hod = """
SELECT ?vav
WHERE {
    ?vav rdf:type brick:VAV .
};
"""

query0_rdf3x = query0_sparql

query1_sparql = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX brick: <http://buildsys.org/ontologies/Brick#> 
PREFIX bf: <http://buildsys.org/ontologies/BrickFrame#> 
SELECT DISTINCT ?sensor ?room
WHERE {

    ?sensor rdf:type/rdfs:subClassOf* brick:Zone_Temperature_Sensor .
    ?room rdf:type brick:Room .
    ?vav rdf:type brick:VAV .
    ?zone rdf:type brick:HVAC_Zone .

    ?vav bf:feeds+ ?zone .
    ?zone bf:hasPart ?room .

    {?sensor bf:isPointOf ?vav }
    UNION
    {?sensor bf:isPointOf ?room }
}
"""
query1_hod = """
SELECT ?sensor ?room
WHERE {
    ?sensor rdf:type/rdfs:subClassOf* brick:Zone_Temperature_Sensor .
    ?room rdf:type brick:Room .
    ?vav rdf:type brick:VAV .
    ?zone rdf:type brick:HVAC_Zone .

    ?vav bf:feeds+ ?zone .
    ?zone bf:hasPart ?room .

    { ?sensor bf:isPointOf ?vav .
    OR
    ?sensor bf:isPointOf ?room . }
};
"""
query1_rdf3x = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX brick: <http://buildsys.org/ontologies/Brick#> 
PREFIX bf: <http://buildsys.org/ontologies/BrickFrame#> 
SELECT DISTINCT ?sensor ?room
WHERE {

    ?sensor rdf:type brick:ZoneTemperatureSensor .
    ?room rdf:type brick:Room .
    ?vav rdf:type brick:VAV .
    ?zone rdf:type brick:HVACZone .

    ?vav bf:feeds ?zone .
    ?zone bf:hasPart ?room .
    ?sensor bf:isPointOf ?vav .
}
"""

query2_sparql = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX brick: <http://buildsys.org/ontologies/Brick#> 
PREFIX bf: <http://buildsys.org/ontologies/BrickFrame#> 
SELECT ?vlv_cmd ?vav
WHERE {
    {
      { ?vlv_cmd rdf:type brick:Reheat_Valve_Command }
      UNION
      { ?vlv_cmd rdf:type brick:Cooling_Valve_Command }
    }
    ?vav rdf:type brick:VAV .
    ?vav bf:hasPoint+ ?vlv_cmd .
}
"""
query2_hod = """
SELECT ?vlv_cmd ?vav
WHERE {
    {
      { 
      ?vlv_cmd rdf:type brick:Reheat_Valve_Command .
      OR
      ?vlv_cmd rdf:type brick:Cooling_Valve_Command .
      }
    }
    ?vav rdf:type brick:VAV .
    ?vav bf:hasPoint+ ?vlv_cmd .
};
"""

query3_sparql = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX brick: <http://buildsys.org/ontologies/Brick#> 
PREFIX bf: <http://buildsys.org/ontologies/BrickFrame#> 
SELECT ?floor ?room ?zone
WHERE {
    ?floor rdf:type brick:Floor .
    ?room rdf:type brick:Room .
    ?zone rdf:type brick:HVAC_Zone .

    ?room bf:isPartOf+ ?floor .
    ?room bf:isPartOf+ ?zone .
}
"""
query3_hod = """
SELECT ?floor ?room ?zone
WHERE {
    ?floor rdf:type brick:Floor .
    ?room rdf:type brick:Room .
    ?zone rdf:type brick:HVAC_Zone .

    ?room bf:isPartOf+ ?floor .
    ?room bf:isPartOf+ ?zone .
};
"""

In [4]:
def run(func, show_first=False, iterations=NUM_ITERATIONS):
    query_times = []
    for i in range(iterations):
        t1 = time.time()*1000
        resp = func()
        t2 = time.time()*1000
        if resp is None: break
        if i == 0 and show_first: print resp.content
        query_times.append(t2-t1)
        print i, '{0:.2f}'.format(t2-t1)
        time.sleep(.5)
    query_avg = avg(query_times)
    print "Took {0:.2f}ms".format(query_avg)

## Hod Benchmark

In [ ]:
# Hod Configs
HOD_SERVER = "http://localhost:47808/query"

In [ ]:
# run hod
!cd hod; ./runserver.sh ; cd ..
#!cd hod; docker kill hod ; docker rm hod; docker run -d --name hod -p47808:47808 gtfierro/hod

In [ ]:
## query 0
def hod_query_0():
    resp = requests.post(HOD_SERVER, data=query0_hod)
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp

run(hod_query_0)

In [ ]:
## query 1https://github.com/neo4j-contrib/sparql-plugin
def hod_query_1():
    resp = requests.post(HOD_SERVER, data=query1_hod)
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp

run(hod_query_1)

In [ ]:
## query 2
def hod_query_2():
    resp = requests.post(HOD_SERVER, data=query2_hod)
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp
run(hod_query_2)

In [ ]:
## query 3
def hod_query_3():
    resp = requests.post(HOD_SERVER, data=query3_hod)
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp
run(hod_query_3)

## Fuseki Benchmark

In [5]:
# Fuseki Configs
FUSEKI_SERVER = "http://localhost:3031/berkeley/query"

In [12]:
!cd fuseki; ./runserver.sh ; cd ..

+ docker pull gtfierro/fuseki
Using default tag: latest
latest: Pulling from gtfierro/fuseki
Digest: sha256:258ffe22f17a04e7792b8fedcb44f4b5b62df627958048f463d99ad26304b7d2
Status: Image is up to date for gtfierro/fuseki:latest
+ docker kill fuseki
Error response from daemon: Cannot kill container fuseki: Container ecdb3b16a2ed635f10b8a492bc2c6652ea1bb59937bb38104565219aa7974324 is not running
+ docker rm fuseki
fuseki
+ docker run -d --name fuseki -p3031:3030 gtfierro/fuseki
0a84b11090cdd546a3e112c698c8b528a7b3c37568caba1886f350855c77e714


In [14]:
## query 0
def fuseki_query_0():
    resp = requests.post(FUSEKI_SERVER, params={'query':query0_sparql})
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp
run(fuseki_query_0)

0 168.97
1 30.31
2 22.44
3 18.85
4 21.25
5 21.27
6 18.86
7 17.91
8 14.02
9 16.64
Took 35.05ms


In [15]:
## query 1
def fuseki_query_1(iterations=1):
    resp = requests.post(FUSEKI_SERVER, params={'query':query1_sparql})
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp
run(fuseki_query_1)

KeyboardInterrupt: 

In [ ]:
## query 2
def fuseki_query_2(iterations=1):
    resp = requests.post(FUSEKI_SERVER, params={'query':query2_sparql})
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp
run(fuseki_query_2, True)

## Alegrograph Benchmark

In [ ]:
ALEGRO_SERVER = "http://localhost:10035/repositories/berkeley/sparql"
auth = HTTPBasicAuth('root','asdfasdf')  

In [ ]:
!cd alegrograph; ./runserver.sh ; cd ..

In [ ]:
def alegro_query_0():
    resp = requests.post(ALEGRO_SERVER, data={'query': query0_sparql}, auth=auth)
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp
run(alegro_query_0)

In [ ]:
def alegro_query_1():
    resp = requests.post(ALEGRO_SERVER, data={'query': query1_sparql}, auth=auth)
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp
run(alegro_query_1)

In [ ]:
def alegro_query_2():
    resp = requests.post(ALEGRO_SERVER, data={'query': query2_sparql}, auth=auth)
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp
run(alegro_query_2)

In [ ]:
def alegro_query_3():
    resp = requests.post(ALEGRO_SERVER, data={'query': query3_sparql}, auth=auth)
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp
run(alegro_query_3)

## RDF3X Benchmark

In [16]:
RDF3X_SERVER = "http://localhost:8080/bar"

In [17]:
!cd rdf3x; ./runserver.sh ; cd ..

+ docker pull gtfierro/rdf3x
Using default tag: latest
latest: Pulling from gtfierro/rdf3x
Digest: sha256:d2e7690aaf506da2a85b69c4370e9809f7d3a94c53bb99ffea60ed06a29a7ca6
Status: Image is up to date for gtfierro/rdf3x:latest
+ docker kill rdf3x
rdf3x
+ docker rm rdf3x
rdf3x
+ docker run -d --name rdf3x -p8080:8080 gtfierro/rdf3x
5374b0e1a94ad06cf7720ac48a17fa6251735f76de149037686de803964f277c


In [27]:
def rdf3x_query_0():
    resp = requests.post(RDF3X_SERVER, data=query0_rdf3x)
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp
run(rdf3x_query_0)

0 9.16
1 9.82
2 8.18
3 9.31
4 6.40
5 6.62
6 6.12
7 9.02
8 6.97
9 8.83
Took 8.04ms


In [ ]:
def rdf3x_query_1():
    resp = requests.post(RDF3X_SERVER, data=query1_rdf3x)
    if not resp.ok:
        print resp, resp.reason
        return None
    print resp.content
    return resp
run(rdf3x_query_1)

<empty result>

0 5.66
<empty result>

1 6.57
<empty result>

2 6.22
<empty result>

3 6.04
<empty result>

4 5.83
<empty result>

5 6.36
<empty result>

6 5.59
<empty result>
